In [1]:
import os
import pandas as pd
from shutil import copy2

def convert_to_yolo(row, width, height):
    x1 = row['Roi.X1']
    y1 = row['Roi.Y1']
    x2 = row['Roi.X2']
    y2 = row['Roi.Y2']

    box_width = x2 - x1
    box_height = y2 - y1
    center_x = x1 + box_width / 2
    center_y = y1 + box_height / 2

    center_x /= width
    center_y /= height
    box_width /= width
    box_height /= height

    return center_x, center_y, box_width, box_height

def process_csv(csv_path, images_root_dir, output_images_dir, output_labels_dir):
    data_csv = pd.read_csv(csv_path)
    for index, row in data_csv.iterrows():
        image_path = row['Path']
        image_name = os.path.basename(image_path)
        txt_name = image_name.replace('.png', '.txt')

        width, height = row['Width'], row['Height']
        center_x, center_y, box_width, box_height = convert_to_yolo(row, width, height)
        cls = row['ClassId']

        src = os.path.join(images_root_dir, image_path)
        dst = os.path.join(output_images_dir, image_name)

        os.makedirs(output_images_dir, exist_ok=True)

        if os.path.exists(src):
            copy2(src, dst)
        else:
            continue

        label_path = os.path.join(output_labels_dir, txt_name)
        os.makedirs(output_labels_dir, exist_ok=True)
        with open(label_path, 'w') as f:
            f.write(f'{cls} {center_x} {center_y} {box_width} {box_height}\n')

train_csv_path = '../input/gtsrb-german-traffic-sign/Train.csv'
test_csv_path = '../input/gtsrb-german-traffic-sign/Test.csv'
images_root_dir = '../input/gtsrb-german-traffic-sign'  
output_train_images_dir = 'datasets/train/images'
output_train_labels_dir = 'datasets/train/labels'
output_test_images_dir = 'datasets/val/images'
output_test_labels_dir = 'datasets/val/labels'

process_csv(train_csv_path, images_root_dir, output_train_images_dir, output_train_labels_dir)
process_csv(test_csv_path, images_root_dir, output_test_images_dir, output_test_labels_dir)


In [2]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import numpy as np
import albumentations as A
import random
import re

def load_yolo_annotations(txt_path):
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        center_x = float(parts[1])
        center_y = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        annotations.append([class_id, center_x, center_y, width, height])
    return annotations

def save_yolo_annotations(txt_path, annotations):
    with open(txt_path, 'w') as f:
        for ann in annotations:
            f.write(f"{ann[0]} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n")

def convert_yolo_to_albumentations(size, annotations):
    height, width = size
    albumentations_bboxes = []
    for ann in annotations:
        class_id, center_x, center_y, bbox_width, bbox_height = ann
        x_min = (center_x - bbox_width / 2) * width
        y_min = (center_y - bbox_height / 2) * height
        x_max = (center_x + bbox_width / 2) * width
        y_max = (center_y + bbox_height / 2) * height
        albumentations_bboxes.append([x_min, y_min, x_max, y_max, class_id])
    return albumentations_bboxes

def convert_albumentations_to_yolo(size, albumentations_bboxes):
    height, width = size
    yolo_annotations = []
    for bbox in albumentations_bboxes:
        x_min, y_min, x_max, y_max, class_id = bbox
        center_x = (x_min + x_max) / 2 / width
        center_y = (y_min + y_max) / 2 / height
        bbox_width = (x_max - x_min) / width
        bbox_height = (y_max - y_min) / height
        yolo_annotations.append([class_id, center_x, center_y, bbox_width, bbox_height])
    return yolo_annotations

def augment_image_and_bboxes(image, bboxes):
    height, width = image.shape[:2]
    zoom_out_transform = A.Compose([
        A.Resize(height=height//2, width=width//2, interpolation=cv2.INTER_LINEAR, p=1),
        A.PadIfNeeded(min_height=height, min_width=width, border_mode=cv2.BORDER_CONSTANT, value=(0, 0, 0), p=1)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    transformed = zoom_out_transform(image=image, bboxes=bboxes, class_labels=[bbox[4] for bbox in bboxes])
    return transformed['image'], transformed['bboxes']

def process_image_and_annotations(image_path, txt_path):
    print(f"Processing {image_path} and {txt_path}")
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to read image file {image_path}. Skipping this file.")
        return

    annotations = load_yolo_annotations(txt_path)
    
    albumentations_bboxes = convert_yolo_to_albumentations(image.shape[:2], annotations)
    augmented_image, augmented_bboxes = augment_image_and_bboxes(image, albumentations_bboxes)
    updated_annotations = convert_albumentations_to_yolo(augmented_image.shape[:2], augmented_bboxes)
    
    cv2.imwrite(image_path, augmented_image)
    save_yolo_annotations(txt_path, updated_annotations)

def process_directory(images_dir, labels_dir, is_train):
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]

    class_files = {}
    for label_file in label_files:
        if is_train:
            try:
                class_id = int(label_file.split('_')[0])
                if class_id not in class_files:
                    class_files[class_id] = []
                class_files[class_id].append(label_file)
            except ValueError:
                print(f"Skipping file {label_file} due to naming issues.")
                continue
        else:
            class_id = 'val'
            if class_id not in class_files:
                class_files[class_id] = []
            class_files[class_id].append(label_file)

    for class_id, files in class_files.items():
        num_files_to_augment = int(len(files) * 0.4)
        files_to_augment = random.sample(files, num_files_to_augment)
        
        for file in files_to_augment:
            base_filename = os.path.splitext(file)[0]
            image_path = os.path.join(images_dir, base_filename + '.png') 
            if not os.path.exists(image_path):
                image_path = os.path.join(images_dir, base_filename + '.jpg') 
            
            txt_path = os.path.join(labels_dir, file)
            
            if os.path.exists(image_path):
                process_image_and_annotations(image_path, txt_path)
            else:
                print(f"Image file {image_path} not found for {file}, skipping this file.")

# Directory paths
train_images_dir = '/kaggle/working/datasets/train/images'
train_labels_dir = '/kaggle/working/datasets/train/labels'
val_images_dir = '/kaggle/working/datasets/val/images'
val_labels_dir = '/kaggle/working/datasets/val/labels'

# Process directories
process_directory(train_images_dir, train_labels_dir, is_train=True)
process_directory(val_images_dir, val_labels_dir, is_train=False)


Processing /kaggle/working/datasets/train/images/00001_00045_00019.png and /kaggle/working/datasets/train/labels/00001_00045_00019.txt
Processing /kaggle/working/datasets/train/images/00001_00036_00005.png and /kaggle/working/datasets/train/labels/00001_00036_00005.txt
Processing /kaggle/working/datasets/train/images/00001_00032_00000.png and /kaggle/working/datasets/train/labels/00001_00032_00000.txt
Processing /kaggle/working/datasets/train/images/00001_00017_00021.png and /kaggle/working/datasets/train/labels/00001_00017_00021.txt
Processing /kaggle/working/datasets/train/images/00001_00064_00017.png and /kaggle/working/datasets/train/labels/00001_00064_00017.txt
Processing /kaggle/working/datasets/train/images/00001_00060_00019.png and /kaggle/working/datasets/train/labels/00001_00060_00019.txt
Processing /kaggle/working/datasets/train/images/00001_00052_00027.png and /kaggle/working/datasets/train/labels/00001_00052_00027.txt
Processing /kaggle/working/datasets/train/images/00001_

In [4]:
import os
import yaml

def create_yolo8_yaml(train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, yaml_output_path):
    data = {
        'train': train_images_dir,
        'val': val_images_dir,
        'nc': len(os.listdir(train_labels_dir)),  # Number of classes (assuming one label file per class)
        'names': [str(i) for i in range(len(os.listdir(train_labels_dir)))]  # Assuming class names are integers
    }
    
    with open(yaml_output_path, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

# Directory paths
train_images_dir = '/kaggle/working/datasets/train/images'
train_labels_dir = '/kaggle/working/datasets/train/labels'
val_images_dir = '/kaggle/working/datasets/val/images'
val_labels_dir = '/kaggle/working/datasets/val/labels'
yaml_output_path = 'datasets/data.yaml'

create_yolo8_yaml(train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, yaml_output_path)


In [5]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [6]:
!nvidia-smi

Tue Jun 18 13:19:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# !yolo detect mode=train data=/kaggle/input/data-yaml/data.yaml model=yolov8s.yaml epochs=20 imgsz=512 batch=64

In [8]:
# !yolo detect train data=/kaggle/input/data-yaml/data.yaml model=yolov8n.pt epochs=20 imgsz=800 batch=64


In [9]:
import os
from ultralytics import YOLO

data_path = '/kaggle/input/data-yaml/data.yaml'
model_path = 'yolov8s.pt'

model = YOLO(model_path)

model.train(
    data=data_path,
    epochs=40,
    imgsz=640,
    batch=64,
    fliplr=0.0,    # Hindari flipping horizontal (mirroring)
    flipud=0.0,    # Hindari flipping vertikal (mirroring)
    degrees=0.0,   # Hindari rotasi
)


100%|██████████| 21.5M/21.5M [00:00<00:00, 191MB/s]
New https://pypi.org/project/ultralytics/8.2.35 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/input/data-yaml/data.yaml, epochs=40, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78c389352560>
fitness: 0.9702286727148302
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.97391,     0.96566,     0.97755,     0.97926,     0.97577,     0.95231,     0.96993,     0.96487,     0.95826,     0.97098,     0.97103,     0.97363,     0.97249,     0.98358,     0.98143,     0.96954,     0.97825,     0.94403,     0.96595,     0.97642,      0.9416,     0.96021,     0.95673,     0.97903,
           0.98614,     0.97484,     0.95681,     0.98092,     0.97539,     0.97655,     0.97339,     0.97878,     0.94366,     0.95857,     0.95547,     0.96624,